In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import os
import time
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/RNN with fortune/fortunes.csv"
data = pd.read_csv(path)
data = data.drop(columns=['Unnamed: 1']).dropna()
data = data.sample(frac=1).reset_index(drop=True)
print(len(data))
data.head(3)

12946


,Fortune
0,You may be tempted to interfere in someone els...
1,Because Taurus is one of the zodiac’s “fixed” ...
2,People will be demanding of you today and ther...


In [4]:
raw_train_ds = tf.data.Dataset.from_tensor_slices(data["Fortune"])
next(iter(raw_train_ds))

<tf.Tensor: shape=(), dtype=string, numpy=b'You may be tempted to interfere in someone else\xe2\x80\x99s personal life today but the planets warn that nothing good will come of it, so resist the urge and keep your distance. The simple fact is you cannot help everyone, nor should you want to.'>

In [5]:
text_dataset = tf.data.Dataset.from_tensor_slices(data["Fortune"])
max_features = 10000  # Maximum vocab size.
max_len = 150  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len,
 standardize=None)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
vectorize_layer.adapt(text_dataset.batch(64))

In [6]:
before = vectorize_layer.get_vocabulary()
print(before[:5])
after = before.copy()
after.remove("[UNK]")
print(after[:5])

['', '[UNK]', 'you', 'to', 'the']
['', 'you', 'to', 'the', 'and']


In [7]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)
print(vectorize_text(next(iter(raw_train_ds))))
print(next(iter(raw_train_ds)))

tf.Tensor(
[[  18   23    8  388    3 2827   14   94 1644  405   82   42   16    4
    93  202   12  175   56   10  105    7  416   25  913    4  553    5
   131    6 2069   37  465  183   11    2  179  170 2235 1468   76    2
    80  569    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]], shape=(1, 150), dtype=int64)
tf.Tensor(b'You may be tempted to interfere in someone else\xe2\x80\x99s personal life today but the planets warn that nothing good will come of it, so resist the urge and keep your dis

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=after)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=after, invert=True)
ids_from_chars.get_vocabulary()[:5]

['[UNK]', '', 'you', 'to', 'the']

In [9]:
chars_sample = chars_from_ids(vectorize_text(next(iter(raw_train_ds))))
print(chars_sample)

tf.Tensor(
[[b'You' b'may' b'be' b'tempted' b'to' b'interfere' b'in' b'someone'
  b'else\xe2\x80\x99s' b'personal' b'life' b'today' b'but' b'the'
  b'planets' b'warn' b'that' b'nothing' b'good' b'will' b'come' b'of'
  b'it,' b'so' b'resist' b'the' b'urge' b'and' b'keep' b'your'
  b'distance.' b'The' b'simple' b'fact' b'is' b'you' b'cannot' b'help'
  b'everyone,' b'nor' b'should' b'you' b'want' b'to.' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[U

In [10]:
ids_from_chars(chars_sample)

<tf.Tensor: shape=(1, 150), dtype=int64, numpy=
array([[  18,   23,    8,  388,    3, 2827,   14,   94, 1644,  405,   82,
          42,   16,    4,   93,  202,   12,  175,   56,   10,  105,    7,
         416,   25,  913,    4,  553,    5,  131,    6, 2069,   37,  465,
         183,   11,    2,  179,  170, 2235, 1468,   76,    2,   80,  569,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [12]:
ids_from_chars.get_vocabulary()[94]

'someone'

In [13]:
chars_sample = chars_from_ids(vectorize_text(next(iter(raw_train_ds))))
print(chars_sample)

tf.Tensor(
[[b'You' b'may' b'be' b'tempted' b'to' b'interfere' b'in' b'someone'
  b'else\xe2\x80\x99s' b'personal' b'life' b'today' b'but' b'the'
  b'planets' b'warn' b'that' b'nothing' b'good' b'will' b'come' b'of'
  b'it,' b'so' b'resist' b'the' b'urge' b'and' b'keep' b'your'
  b'distance.' b'The' b'simple' b'fact' b'is' b'you' b'cannot' b'help'
  b'everyone,' b'nor' b'should' b'you' b'want' b'to.' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[U

In [14]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds
ds = raw_train_ds.map(vectorize_text)


In [15]:
#ds is now [[1,4,12,8]]
for i in ds.take(1):
  print(i)

tf.Tensor(
[[  18   23    8  388    3 2827   14   94 1644  405   82   42   16    4
    93  202   12  175   56   10  105    7  416   25  913    4  553    5
   131    6 2069   37  465  183   11    2  179  170 2235 1468   76    2
    80  569    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]], shape=(1, 150), dtype=int64)


In [16]:
for i in ds.take(1):
  print(chars_from_ids(i))

tf.Tensor(
[[b'You' b'may' b'be' b'tempted' b'to' b'interfere' b'in' b'someone'
  b'else\xe2\x80\x99s' b'personal' b'life' b'today' b'but' b'the'
  b'planets' b'warn' b'that' b'nothing' b'good' b'will' b'come' b'of'
  b'it,' b'so' b'resist' b'the' b'urge' b'and' b'keep' b'your'
  b'distance.' b'The' b'simple' b'fact' b'is' b'you' b'cannot' b'help'
  b'everyone,' b'nor' b'should' b'you' b'want' b'to.' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
  b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[U

In [18]:
def split_input_target(sequence):
    input_text = sequence[0][:-1]
    target_text = sequence[0][1:]
    return input_text, target_text

#create input, output for each sample
dataset = ds.map(split_input_target)
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset, len(data), shuffle=False)

In [19]:
for i in test_ds.take(1):
  print(i)

(<tf.Tensor: shape=(149,), dtype=int64, numpy=
array([  18,   10,   27,    4,  241,    3,  389,   21,    2,   38,   28,
         42,    5,    2,   66, 1719,   13,    5,  364,   68,   44,    2,
        487,  275,    9, 2563,  558,    3, 1129,    2,   98,    2,   23,
        423,    4, 3945,    7,    4,  268, 4494,   50, 1084,   19,  191,
         25, 6572,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
    

In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 149), (64, 149)), types: (tf.int64, tf.int64)>

In [21]:
# Length of the vocabulary in chars
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 149, 10000) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()


Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2560000   
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  10250000  
Total params: 16,748,304
Trainable params: 16,748,304
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([3115, 7158, 4688, 6540, 4049, 7120, 5175, 6229, 9713, 6995, 5414,
       4747, 2046, 7084, 2122, 9182, 6685, 8580, 5914,  812, 1901, 9380,
       9082, 5003, 6371,  764,  968, 7373, 8304, 1413, 6818, 9217, 5665,
       5759, 1417, 3179, 9981, 1069, 1631, 2182, 7517, 4150, 5502, 4096,
       8539, 3021,  361,  505, 3218, 8607,  293, 5718, 1366, 4367, 1030,
       8320, 6841, 3998, 6409,  717, 9705, 2338, 7270, 4803, 5067, 1969,
       7597, 5885,   52,  704,   75, 9099, 8579, 3189, 1301, 4905, 2005,
       5411, 1539, 4984, 8574, 9142, 9818, 9728, 2963, 4472, 9701, 2780,
        380, 4480, 9462, 4264, 4461, 3015, 3695, 1247, 8738, 2644, 3309,
       6047, 1903, 2166, 7881, 6295, 8010, 4606, 2287, 7670, 5198, 8460,
        309, 5673, 2396, 2506,  185, 4502, 4670, 1563, 7320, 2847, 7263,
         88, 9165, 7381, 3157, 5520, 6902, 2606, 9973, 1780, 3243, 4326,
        974, 9424,  197, 9466, 1832,   10, 4013, 6995, 8256,  264, 3041,
       8173, 2074, 4015,  263, 3563, 5295])

In [28]:
print("Input:\n", chars_from_ids(input_example_batch[0]))
print()
print("Next Char Predictions:\n", chars_from_ids(sampled_indices))

Input:
 tf.Tensor(
[b'Something' b'will' b'happen' b'that' b'catches' b'you' b'by'
 b'surprise' b'over' b'the' b'next' b'24' b'hours.' b'It' b'may' b'not'
 b'be' b'the' b'kind' b'of' b'surprise' b'you' b'enjoy' b'very' b'much'
 b'but' b'if' b'you' b'are' b'honest' b'you' b'will' b'admit,' b'if'
 b'only' b'to' b'yourself,' b'that' b'you' b'needed' b'a' b'shock' b'to'
 b'your' b'system.' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]'
 b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' b'[UNK]' 

In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 149, 10000)  # (batch_size, sequence_length, vocab_size)
Mean loss:         9.207552


In [31]:
tf.exp(mean_loss).numpy()

9972.154

In [32]:

model.compile(optimizer='adam', loss=loss)

In [33]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
202/202 [==============================] - 105s 514ms/step - loss: 0.2741
Epoch 2/5
202/202 [==============================] - 105s 510ms/step - loss: 0.2497
Epoch 3/5
202/202 [==============================] - 105s 510ms/step - loss: 0.2280
Epoch 4/5
202/202 [==============================] - 106s 515ms/step - loss: 0.2085
Epoch 5/5
202/202 [==============================] - 105s 513ms/step - loss: 0.1908


In [71]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs. ["HK HSBC"] -> ["HK","HSBC"]
    x = tf.strings.split(
    inputs, sep=" ", maxsplit=-1, name=None)
    #["HK","HSBC"] -> [14,15]
    input_ids = ids_from_chars(x).to_tensor()
    #print(input_ids)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [121]:
import time
start = time.time()
states = None
next_char = ["Try"]
result = [next_char]
#print(len(next_char[0].split()))
end = 0
for n in range(50):
  if end == 0:
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    try: 
      if next_char.numpy()[0].decode('utf-8')[-1] == ".":
        end = 1
    except:
      print(next_char.numpy()[0].decode('utf-8'))
      end = 1
  #print(next_char.numpy()[0].decode('utf-8'))
result = tf.strings.join(result,separator=' ')
end = time.time()
#print(result[0])
#print('\nRun time:', end - start)
result.numpy()[0].decode('utf-8')


'Try not to get impatient with people and do whatever it takes to get along with them today.'

In [117]:
#os.listdir(checkpoint_dir)

In [58]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'./training_checkpoints/ckpt_5'

In [59]:
# Save the weights
model.save_weights('/content/drive/MyDrive/Colab Notebooks/RNN with fortune/my_check_point')



### Reusing model

In [61]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
#create new model
new_mod = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
new_mod.compile(optimizer='adam', loss=loss)
# Restore the weights
new_mod.load_weights('/content/drive/MyDrive/Colab Notebooks/RNN with fortune/my_check_point')
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs. ["HK HSBC"] -> ["HK","HSBC"]
    x = tf.strings.split(
    inputs, sep=" ", maxsplit=-1, name=None)
    #["HK","HSBC"] -> [14,15]
    input_ids = ids_from_chars(x).to_tensor()
    print(input_ids)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states
one_step_model = OneStep(new_mod, chars_from_ids, ids_from_chars)


In [62]:
import time
start = time.time()
states = None
next_char = ["Why is it"]
result = [next_char]
print(len(next_char[0].split()))
for n in range(50):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)
  #print(next_char)
result = tf.strings.join(result,separator=' ')
end = time.time()
print(result[0])
print('\nRun time:', end - start)

3
Tensor("RaggedToTensor/RaggedTensorToTensor:0", shape=(1, None), dtype=int64)
Tensor("RaggedToTensor/RaggedTensorToTensor:0", shape=(1, None), dtype=int64)
Tensor("RaggedToTensor/RaggedTensorToTensor:0", shape=(None, None), dtype=int64)
tf.Tensor(b'Why is it that you intend to do so much about what might  \xe2\x80\x93 so long as the year is  things in yourself? No matter where it may be that certain people are being annoyed over the course and it\xe2\x80\x99s the only way you will have to go along the flow.', shape=(), dtype=string)

Run time: 1.7247834205627441
